**Google BigQuery I/O connector**
- reference link: https://beam.apache.org/documentation/io/built-in/google-bigquery/
- content: we will discuss about bigquery class and object in apache beam python

**Bigquery Basics**
- Table names
    - using a string: [project_id]:[dataset_id].[table_id]
    - using a TableReference
- Table rows
    - BigQueryIO read and write transforms produce and consume data as a PCollection of dictionaries, where each element in the PCollection represents a single row in the table.
    - a PCollection is a collection of each element is a dictionary. Each dictionary is a row in bigquery table
- Schemas
    - When writing to BigQuery, you must supply a table schema for the destination table that you want to write to
- Data types
    - reference link: https://beam.apache.org/documentation/io/built-in/google-bigquery/#data-types


**Create table schema**
- using a TableSchema
- using a string
- referenc link: https://beam.apache.org/documentation/io/built-in/google-bigquery/#creating-a-table-schema


In [ ]:
from decimal import *
import base64
#create schema using talbe schema
    #python present TableSchem object with dictionary
table_schema = {
    'fields': [{
        'name': 'source', 'type': 'STRING', 'mode': 'NULLABLE'
    }, {
        'name': 'quote', 'type': 'STRING', 'mode': 'REQUIRED'
    }]
}
#present table schem with string with format: “field1:type1,field2:type2,field3:type3” 
table_schema = 'source:STRING, quote:STRING'

#bigquery data type and data presentation in bigquery
bigquery_data = [{
    'string': 'abc',
    'bytes': base64.b64encode(b'\xab\xac'),
    'integer': 5,
    'float': 0.5,
    'numeric': Decimal('5'),
    'boolean': True,
    'timestamp': '2018-12-31 12:44:31.744957 UTC',
    'date': '2018-12-31',
    'time': '12:44:31',
    'datetime': '2018-12-31T12:44:31',
    'geography': 'POINT(30 10)'
}]

**Read from BigQuery**
- read from a table
- read from a query
- using storage read api. Python sdk still not supporting it
- reference link: https://beam.apache.org/documentation/io/built-in/google-bigquery/#reading-from-bigquery

In [ ]:
#example of read data from bigquery with a query
import apache_beam as beam
max_temperatures = (
    pipeline
    | 'QueryTableStdSQL' >> beam.io.ReadFromBigQuery(
        query='SELECT max_temperature FROM '\
              '`clouddataflow-readonly.samples.weather_stations`',
        use_standard_sql=True)
    # Each row is a dictionary where the keys are the BigQuery columns
    | beam.Map(lambda elem: elem['max_temperature']))

**Writing to Bigquery**
When you apply a write transform, you must provide the following information for the destination table(s):

- The table name.
- The destination table’s create disposition. The create disposition specifies whether the destination table must exist or can be created by the write operation.
- The destination table’s write disposition. The write disposition specifies whether the data you write will replace an existing table, append rows to an existing table, or write only to an empty table.

**Create disposition**
- decide whether or not to create table when writing transform
- BigQueryDisposition.CREATE_IF_NEEDED: create table when table is not exist
- BigQueryDisposition.CREATE_NEVER: never create table


**Write disposition**
- attribute to control how to write to an existing table
- BigQueryDisposition.WRITE_EMPTY
- BigQueryDisposition.WRITE_TRUNCATE
- BigQueryDisposition.WRITE_APPEND

In [ ]:
#example of writing data to a table
    #1 specify table
    #2 specify schem
    #3 specify create_disposition and write_disposition
quotes = pipeline | beam.Create([
    {
        'source': 'Mahatma Gandhi', 'quote': 'My life is my message.'
    },
    {
        'source': 'Yoda', 'quote': "Do, or do not. There is no 'try'."
    },
])
quotes | beam.io.WriteToBigQuery(
    table_spec,
    schema=table_schema,
    write_disposition=beam.io.BigQueryDisposition.WRITE_TRUNCATE,
    create_disposition=beam.io.BigQueryDisposition.CREATE_IF_NEEDED)

In [ ]:
#Example: Using dynamic destinations
fictional_characters_view = beam.pvalue.AsDict(
    pipeline | 'CreateCharacters' >> beam.Create([('Yoda', True),
                                                  ('Obi Wan Kenobi', True)]))

def table_fn(element, fictional_characters):
  if element in fictional_characters:
    return 'my_dataset.fictional_quotes'
  else:
    return 'my_dataset.real_quotes'

quotes | 'WriteWithDynamicDestination' >> beam.io.WriteToBigQuery(
    table_fn,
    schema=table_schema,
    table_side_inputs=(fictional_characters_view, ),
    write_disposition=beam.io.BigQueryDisposition.WRITE_TRUNCATE,
    create_disposition=beam.io.BigQueryDisposition.CREATE_IF_NEEDED)

In [ ]:
#example of specify partitioni when writing table
quotes | 'WriteWithTimePartitioning' >> beam.io.WriteToBigQuery(
    table_spec,
    schema=table_schema,
    write_disposition=beam.io.BigQueryDisposition.WRITE_TRUNCATE,
    create_disposition=beam.io.BigQueryDisposition.CREATE_IF_NEEDED,
    additional_bq_parameters={'timePartitioning': {
        'type': 'HOUR'
    }})

**Setting the insertion method**
- there are two insert method:
    - FILE_LOADS
    - STREAMING_INSERTS
- note with file loads method: with files load method we have to specif temp bucket (data is stored in temp bucket before write to bigquery table).

In [ ]:
#Example of specify insert method: 
WriteToBigQuery(method='FILE_LOADS')
WriteToBigQuery(method='STREAMING_INSERTS')